In [1]:
# Packages
import os
import csv
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from math import ceil
from googletrans import Translator
import pycountry

### Data importation

In [3]:
df = pd.read_excel("EuropeTop100Attractions_ENG_20190101_20210821.xlsx", sheet_name='Reviews', engine="openpyxl")

### Functions

In [33]:
geolocator = Nominatim(user_agent="geo_locator")

def get_iso_code(location):
    try:
        iso_code = pycountry.countries.search_fuzzy(location)[0].alpha_2
    except (LookupError, IndexError):
        try:
            geocode_result = geolocator.geocode(location, language='en')
            iso_code = next((country.alpha_2 for part in geocode_result.raw.get('display_name', '').split(', ')[::-1]
                            if (country := pycountry.countries.get(name=part))), None)
        except Exception as e:
            iso_code = None

    return iso_code

# Data Cleaning: UserLocation
##### Data Cleaning Process for User Locations
To clean the user locations in our dataset, we initially attempted to use the geopy.geocoders package. However, due to the sheer volume of diverse locations, this approach encountered challenges in returning the results for all locations. As a result, we incorporated the pycountry package to improve the efficiency of the process.

While pycountry significantly accelerates the geocoding process, it is not infallible, and certain limitations exist. For instance, when extracting ISO codes for locations with state abbreviations following a comma, such as 'Indianapolis, IN', inaccuracies may arise, with the ISO code erroneously corresponding to other countries, as in the case of 'Indianapolis, IN' being misinterpreted as India.

Given these challenges and to streamline the process for a more manageable control, we adopted the following approach:

1. Initial setup
We begin by extracting and working with unique values from the userLocation column. This allows us to focus on a more manageable set during the normalization process.

2. Geocoding Locations
Subsequently, we employ the geocoding code. This step allows us to systematically address the entire dataset, identifying and rectifying any geocoding errors that may arise. In the end we can adjust some exceptiosn almost 'manually' beacause they are significally less (322 locations) and we can controle it btter. 

3. Verify the values in excel
After geocoding and adjustments, the results are consolidated and verified using Excel.

In [4]:
df.head()

,localID,extractionDate,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewLanguage,reviewFullText
0,MAG001,2021-08-20 08:24:40.077,4.5,1,1186,163828,Maurice M@MauriceM171,"Indianapolis, IN",29,Couples,2021-08-18,2021-08-01,5,en,"Absolutely breathtaking use of light, an amazi..."
1,MAG001,2021-08-20 08:24:40.097,4.5,1,1186,163828,nickchamb@nickchamb,"Bandon, Ireland",7,Couples,2021-08-17,2021-03-01,5,en,A really interesting and beautiful place to vi...
2,MAG001,2021-08-20 08:24:40.117,4.5,1,1186,163828,Arthur Villon@arthur4588,"Valencia, Spain",268,Friends,2021-08-16,2021-08-01,5,en,Beautiful place and must visit place in barcel...
3,MAG001,2021-08-20 08:24:40.147,4.5,1,1186,163828,James U@696jamesu,NaN,7,NaN,2021-08-16,2021-07-01,5,en,A MUST when visiting Barcelkona. It is consta...
4,MAG001,2021-08-20 08:24:40.159,4.5,1,1186,163828,Kamal K Sirra@Sirra63,"Madrid, Spain",516,Family,2021-08-15,2021-07-01,4,en,I visited about 20 years ago and there has bee...


In [5]:
df.shape

(92120, 15)

In [14]:
# Create a new DataFrame with only the 'userLocation' column
df_user_location = df[['userLocation']].copy()

In [15]:
df_user_location.shape

(92120, 1)

In [16]:
df_user_location.head()

,userLocation
0,"Indianapolis, IN"
1,"Bandon, Ireland"
2,"Valencia, Spain"
3,NaN
4,"Madrid, Spain"


In [17]:
# Drop duplicates and NaN values in the 'userLocation'
df_user_location.drop_duplicates(subset=['userLocation'], inplace=True)
df_user_location.dropna(subset=['userLocation'], inplace=True)

In [18]:
df_user_location.shape

(12613, 1)

In [22]:
df_user_location['userISOCode'] = df_user_location['userLocation'].apply(get_iso_code)

In [23]:
df_user_location.head()

,userLocation,userISOCode
0,"Indianapolis, IN",US
1,"Bandon, Ireland",IE
2,"Valencia, Spain",ES
4,"Madrid, Spain",ES
5,"Towaco, NJ",US


In [24]:
print(df_user_location['userISOCode'].isna().sum())

322


In [26]:
# Set Pandas display options to show all rows and columns
pd.set_option('display.max_rows', None)

missing_values_rows = df_user_location[df_user_location['userISOCode'].isna()]

print(missing_values_rows)

                                         userLocation userISOCode
8                    Sofiyivska Borschagivka, Ukraine        None
40                       Province of Barcelona, Spain        None
122            The Royal Town of Sutton Coldfield, UK        None
192                                    Moscow, Russia        None
444                                Stenloese, Denmark        None
507                       Paso Robles Wine Region, CA        None
702                                      Tehran, Iran        None
777                                           2271269        None
833                                            LexGSR        None
850                                            Europe        None
931                       Saint-Denis, Reunion Island        None
1015                        Ho Chi Minh City, Vietnam        None
1187                           St. Petersburg, Russia        None
1243                            Yekaterinburg, Russia        None
1808      

In [44]:
def get_iso_code2(location):
    try:
        # Check for specific state cases in the US
        us_states = {
            'PA': 'US', 'MN': 'US', 'AL': 'US', 'CA': 'US', 'NY': 'US', 'TX': 'US',
            'FL': 'US', 'VT': 'US', 'NH': 'US', 'GA': 'US', 'HI': 'US'
        }
        if location.upper() in us_states:
            return us_states[location.upper()]

        # Merge all country exceptions into a single dictionary
        country_exceptions = {
            'St-Michel-des-Sts': 'CA', 'Mom in Massachusetts': 'US', 'Biniamina': 'IL',
            'East Covertry': 'US', 'Coloradp': 'US', 'kinston upon hull': 'GB',
            'Nassau, New Providence Island': 'BS', 'Reckange/Meersch': 'LU',
            'New Providence Island': 'BS', 'Southern Califonia': 'US', 'hcm city': 'VN',
            'Hull  East Yorks': 'GB', 'Luang Prabang': 'LA', 'Derrycity Northern Ireland': 'GB',
            'Gtr Manchester': 'GB', 'Mumbassa': 'KE', 'Hanoi': 'VN', 'Guntrausdorf': 'AT',
            'Staffordshsire': 'GB', 'Moscow': 'RU', 'The Cincinnati Region': 'US',
            'Chicago vicinity': 'US', 'Kinshasa, Democratic Republic of the Congo': 'CD',
            'Seattle, the Great Pacific Northwest!': 'US', 'Bodrum, Mugla Province, Turkish Aegean Coast': 'TR',
            'Danville, Callifornia': 'US', 'Star707, Sydney': 'AU', 'Санкт-Петербург, Россия': 'RU',
            "Houston, Texas ya'll": 'US', 'harrogate Yorks': 'GB', 'Happy in Surrey': 'GB',
            'Chcago': 'US', 'Reshon Lezion': 'IL', 'Devpur Kutch': 'IN', 'the Nethlands': 'NL',
            'Nehterlands': 'NL', 'couple from norfolk england': 'GB', 'Driffield, North Humberside': 'GB',
            'Batley, West Yorks': 'GB', 'West Kirby, Glannau Mersi, Deyrnas Unedig.': 'GB',
            'Brufut-Gambia': 'GM', 'Berkhamsted, Hertforshire': 'GB', 'west coast of Scotland': 'GB',
            'Lahain': 'US', 'Chicago - SW burbs': 'US', 'Loondon': 'GB', 'Greater Seattle Area': 'US',
            'New England, U.S.A.': 'US', 'cumnck': 'US', 'Seenoaks': 'GB', 'Beoleil': 'CA',
            'Pinner, Middlesex': 'GB', 'South Coast UK': 'GB', 'Klagenfurt/Križe': 'AT',
            'Norfolk Alien': 'GB', 'South Korea': 'KR', 'Taiwan': 'TW', 'Palestinian Territories': 'PS',
            'Grand Cayman': 'KY', 'Reunion Island': 'RE', 'Puerto Rico': 'PR', 'Russia': 'RU',
            'Cape Verde': 'CV', 'Laos': 'LA', 'Congo': 'CG', 'Syria': 'SY', 'Gambia': 'GM',
            'Scotland': 'GB', 'Kutch': 'IN', 'Netherlands': 'NL', 'England': 'GB', 'Wales': 'GB',
            'Cambridgeshire': 'GB', 'Suriname': 'SR', 'Bonaire': 'BQ', 'French Guiana': 'GF',
            'Falkland Islands': 'FK', 'Curaçao': 'CW', 'Sint Eustatius': 'BQ', 'Saba': 'BQ',
            'Sint Maarten': 'SX', 'Aruba': 'AW', 'Norfolk Island': 'NF', 'Trinidad and Tobago': 'TT',
            'New Zealand': 'NZ', 'Gozo': 'MT', 'Faroe Islands': 'FO', 'Malta': 'MT', 'Greenland': 'GL',
            'Guernsey': 'GG', 'Jersey': 'JE', 'Isle of Man': 'IM'
        }

        # Try to get ISO code directly from pycountry
        iso_code = country_exceptions.get(location, None) or pycountry.countries.search_fuzzy(location)[-1].alpha_2
    except (LookupError, IndexError):
        iso_code = None

    return iso_code

In [45]:
missing = df_user_location['userISOCode'].isna()
df_user_location.loc[missing, 'userISOCode'] = df_user_location.loc[missing, 'userLocation'].apply(get_iso_code2)

In [46]:
print(df_user_location['userISOCode'].isna().sum())

45


In [47]:
missing_values_rows = df_user_location[df_user_location['userISOCode'].isna()]

print(missing_values_rows)

                       userLocation userISOCode
777                         2271269        None
833                          LexGSR        None
850                          Europe        None
2392                    BarnyardGal        None
3281                      Northeast        None
4053                        3218894        None
4427                        2151451        None
4815                     venice0810        None
6498                        BPutnam        None
11687                       3733887        None
12693                        580422        None
12903                      11827142        None
15159                       3203406        None
15345                      14968150        None
16578                        850579        None
19943                          Asia        None
19969                        499520        None
24127                     Baulagirl        None
24728                        298567        None
25187           S30052nellville Ga.     

In [49]:
df_user_location.to_excel("C:\\Users\\catar\\OneDrive\\Documentos\\UserLocationClean.xlsx", index=False)